# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [32]:
import pandas as pd
import numpy as np
from collections import Counter
from scipy import stats

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
<p>1. What test is appropriate for this problem? Does CLT apply?<p>
<p>2. What are the null and alternate hypotheses?<p>
</div>

**Answer**

<p> 1. Random, Independent (4370 samples is < 10% of the population) and Large enough sample size (n = 4870 > 30), therefore Central Limit Theorem applies. <p>

<p> 2. 
- Null hypothesis H_o: Success rates given resumes w/ white- and black- sounding names are the same
- Alternative hypothesis H_a: Success rates given resumes w/ white- and black- sounding names are not the same

In [4]:
# Number of resumes with white-sounding names
w = data[data.race=='w']
len(w)

2435

In [5]:
# Number of resumes with black-sounding names
b = data[data.race=='b']
len(b)

2435

In [9]:
np.sum(w.call)

235.0

In [7]:
# Sample data mean of success rate given white- and black- sounding name
p_w = np.sum(w.call)/len(w)
p_b = np.sum(b.call)/len(b)
p_wb = np.sum(data.call)/len(data)

print(f'Based on the sample data, success rate regardless of race is {round(p_wb,4)}')
print(f'Based on the sample data, success rate given resume with white-sounding names is {round(p_w,4)}')
print(f'Based on the sample data, success rate given resume with black-sounding names is {round(p_b,4)}')

Based on the sample data, success rate regardless of race is 0.0805
Based on the sample data, success rate given resume with white-sounding names is 0.0965
Based on the sample data, success rate given resume with black-sounding names is 0.0645


<div class="span5 alert alert-success">
<p> Your answers to Q3 here </p>
<p> 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches. <p>

</div>

In [31]:
# Chi-squared test
# Null hypothesis H_o: Success rates given resumes w/ white- and black- sounding names are the same --- EXPECTED
# Alternative hypothesis H_a: Success rates given resumes w/ white- and black- sounding names are not the same --- OBSERVED
# Significance level = 0.05

obs_no_callback_w = len(w.call) * (1-p_w)
obs_callback_w = len(w.call) * (p_w)
obs_no_callback_b = len(b.call) * (1-p_b)
obs_callback_b = len(b.call) * (p_b)

exp_no_callback_w = len(w.call) * (1-p_wb)
exp_callback_w = len(w.call) * (p_wb)
exp_no_callback_b = len(b.call) * (1-p_wb)
exp_callback_b = len(b.call) * (p_wb)

x_sq, p = stats.chisquare([obs_no_callback_w,obs_callback_w,obs_no_callback_b,obs_callback_b],[exp_no_callback_w,exp_callback_w,exp_no_callback_b,exp_callback_b],ddof=2)

print (f'Chi-squared is {round(x_sq,3)} and P-value is {round(p,5)} < 0.05. Reject null hypothesis')

Chi-squared is 16.879 and P-value is 4e-05 < 0.05. Reject null hypothesis


In [228]:
# Bootstrap hypothesis test
# Null hypothesis H_o: Success rates given resumes w/ white- and black- sounding names are the same
# Alternative hypothesis H_a: Success rates given resumes w/ white- and black- sounding names are not the same
# Significance level = 0.05

# Compute difference between success rate of resume w/ white-sounding names and w/ black-sounding names
empirical_diff_means = diff_of_means(w.call, b.call)

# Draw 10,000 permutation replicates: perm_replicates
perm_replicates = draw_perm_reps(w.call, b.call, diff_of_means, size=10000)

# Compute p-value: p
p = np.sum(perm_replicates >= empirical_diff_means) / len(perm_replicates)

print (f'P-value of {p} < 0.05. Reject null hypothesis')

P-value of 0.0001 < 0.05. Reject null hypothesis


In [222]:
def draw_perm_reps(data_1, data_2, func, size=1):
    """Generate multiple permutation replicates."""

    # Initialize array of replicates: perm_replicates
    perm_replicates = np.empty(size)

    for i in range(size):
        # Generate permutation sample
        perm_sample_1, perm_sample_2 = permutation_sample(data_1,data_2)

        # Compute the test statistic
        perm_replicates[i] = func(perm_sample_1, perm_sample_2)

    return perm_replicates

def permutation_sample(data1, data2):
    """Generate a permutation sample from two data sets."""

    # Concatenate the data sets: data
    data = np.concatenate((data1,data2))

    # Permute the concatenated array: permuted_data
    permuted_data = np.random.permutation(data)

    # Split the permuted array into two: perm_sample_1, perm_sample_2
    perm_sample_1 = permuted_data[:len(data1)]
    perm_sample_2 = permuted_data[len(data1):]

    return perm_sample_1, perm_sample_2

def diff_of_means(data_1, data_2):
    """Difference in means of two arrays."""

    # The difference of means of data_1, data_2: diff
    diff = np.mean(data_1) - np.mean(data_2)

    return diff

In [237]:
# Alternatively does this work?

# Obtain 100 bootstrap resamples of the original data
bs_num = 100
inds = np.arange(len(data.call))
counter = 0

for n in range(bs_num):

    # Randomly redraw 4870 data points with replacement from sample data
    bs_inds = np.random.choice(inds, size=len(inds))
    store = []
        
    for i in bs_inds:
        store.append([data.iloc[i].race,data.iloc[i].call])

    df_sub = pd.DataFrame(data=store,columns=['race','call'])
    w_sub = df_sub[df_sub.race=='w']
    b_sub = df_sub[df_sub.race=='b']

    # Calculate success rate of resumes given white-sounding and black-sounding names
    w_rate = np.sum(w_sub.call)/len(w_sub.call)
    b_rate = np.sum(b_sub.call)/len(b_sub.call)
    
    # Count +1 if success rate of resumes with black-sounding names is greater than with white-sounding names
    if b_rate > w_rate:
        counter +=1

p = counter / bs_num

print (f'Probability of success rate of resumes with black-sounding names is greater than with white-sounding names is {p}')
print (f'{p} < 0.05. Reject null hypothesis')

Probability of success rate of resumes with black-sounding names is greater than with white-sounding names is 0.0
0.0 < 0.05. Reject null hypothesis


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
<p> 4. Write a story describing the statistical significance in the context or the original problem. <p>
<p> 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis? <p>

</div>

**Answer**

4. The P-value calculated is small and shows that the null hypothesis is significantly different (statistically) than the data.
It is highly unlikely that the success rates between resumes w/ white-sounding names are the same as w/ black-sounding names.

5. The race/name is a factor but one cannot conclude that it is the "most" important factor based on the analysis performed. Other variables (e.g. education, years of experince, etc.) may affect the success rates but are not included in the analysis.